In [1]:
from kafka import KafkaProducer
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

import tkinter as tk

import requests
import json
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession

from bs4 import XMLParsedAsHTMLWarning
import warnings
warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)
warnings.filterwarnings("ignore", message="La méthode frame.append est obsolète et sera supprimée des pandas dans le futur version. Utilisez pandas.concat à la place.")
warnings.filterwarnings("ignore", message="The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.")

In [3]:
#Retourne le code source de l'URL qui est passé en paramètre
def get_source(url):
    try:
        session = HTMLSession()
        response = session.get(url)
        return response

    except requests.exceptions.RequestException as e:
        print(e)

In [4]:
#Retourne un dataFrame qui contient le contenu du flux RSS passé en paramètre
def get_feed(url):   
    response = get_source(url)
    
    df = pd.DataFrame(columns = ['title', 'pubDate', 'link', 'description'])

    with response as r:
        items = r.html.find("item", first=False)

        for item in items:        

            title = item.find('title', first=True).text
            pubDate = item.find('pubDate', first=True).text
            link = item.find('guid', first=True).text
            description = item.find('description', first=True).text


            row = {'title': title, 'pubDate': pubDate, 'link': link, 'description': description}
            df = df.append(row, ignore_index=True)

    return df

In [5]:
#Permet à l'utilisateur de sélectionner les sites internet qu'il veut consulter parmi une liste prédéfinie
#Retourne une liste qui contient ces sites 
def ouvrir_fenetre():
    sites = {
        "Le Monde": "https://www.lemonde.fr/rss/une.xml",
        "Le Figaro": "https://www.lefigaro.fr/rss/figaro_actualites.xml",
        "France Info": "https://www.francetvinfo.fr/titres.rss",
        "Ouest-France": "https://www.ouest-france.fr/rss/une.xml",
        "20 Minutes": "https://www.20minutes.fr/feeds/rss-une.xml",
        "BFM TV": "https://www.bfmtv.com/rss/info/flux-rss/flux-toutes-les-actualites/",
        "France 24": "https://www.france24.com/fr/rss",
        "L'Equipe": "https://www.lequipe.fr/rss/actu_rss_Football.xml",
        "Actu IA": "https://www.actuia.com/feed/",
    }

    def ajouter_site():
        selection.clear()
        for site in listebox.curselection():
            nom_site = listebox.get(site)
            selection.append(sites[nom_site])
        fenetre.quit()

    fenetre = tk.Tk()

    listebox = tk.Listbox(fenetre, selectmode=tk.MULTIPLE)
    for site in sites:
        listebox.insert(tk.END, site)

    bouton_ajouter = tk.Button(fenetre, text="Ajouter", command=ajouter_site)

    selection = []

    listebox.grid(row=0, column=0)
    bouton_ajouter.grid(row=1, column=0)

    fenetre.mainloop()

    return selection

In [6]:
#Appel de la fonction
urls=ouvrir_fenetre()
urls

['https://www.lemonde.fr/rss/une.xml',
 'https://www.lefigaro.fr/rss/figaro_actualites.xml',
 'https://www.francetvinfo.fr/titres.rss',
 'https://www.ouest-france.fr/rss/une.xml',
 'https://www.20minutes.fr/feeds/rss-une.xml',
 'https://www.bfmtv.com/rss/info/flux-rss/flux-toutes-les-actualites/',
 'https://www.france24.com/fr/rss',
 'https://www.lequipe.fr/rss/actu_rss_Football.xml',
 'https://www.actuia.com/feed/']

In [7]:
# Création du dataFrame en fonction des sites sélectionnés
dfs = []
for url in urls:
    df = get_feed(url)
    dfs.append(df)

# Fusion des dataFrame en un seul
data = pd.concat(dfs, ignore_index=True)

In [8]:
#Nettoyage des données 
data['title'] = data['title'].apply(lambda x: x.replace('ê', 'e'))
data['title'] = data['title'].apply(lambda x: x.replace("'", ""))
data['description'] = data['description'].apply(lambda x: x.replace("'", ""))
for i in range(len(data)):
    data['description'] = data['description'].apply(lambda x: x.replace('ê', 'e'))
    if '[' in data.loc[i,'title'] and ']' in data.loc[i,'title']:
        data.loc[i,'title'] = data.loc[i,'title'].split('<![CDATA[')[1].split(']')[0] 
    if '[' in data.loc[i,'description'] and ']' in data.loc[i,'description']:
        data.loc[i,'description'] = data.loc[i,'description'].split('<![CDATA[')[1].split(']')[0] 
    else:
        data.loc[i,'title'] = data.loc[i,'title']
        data.loc[i,'description'] = data.loc[i,'description']

In [ ]:
#Classement des articles par ordre décroissant (plus récents en premier)
data['pubDate'] = pd.to_datetime(data['pubDate'], format='%a, %d %b %Y %H:%M:%S %z')
data = data.sort_values('pubDate', ascending=False)
data = data.reset_index(drop=True)

In [9]:
data

,title,pubDate,link,description
0,,"Fri, 07 Apr 2023 14:35:57 +0200",https://www.lemonde.fr/societe/article/2023/04...,
1,,"Fri, 07 Apr 2023 12:58:00 +0200",https://www.lemonde.fr/politique/article/2023/...,
2,,"Fri, 07 Apr 2023 13:19:43 +0200",https://www.lemonde.fr/international/article/2...,
3,,"Fri, 07 Apr 2023 11:20:10 +0200",https://www.lemonde.fr/international/live/2023...,
4,,"Fri, 07 Apr 2023 06:00:06 +0200",https://www.lemonde.fr/societe/article/2023/04...,
...,...,...,...,...
246,Belgique : European AI Week du 27 mars au 31 mars,"Fri, 24 Mar 2023 14:23:37 +0000",https://www.actuia.com/?p=42834,
247,AWS et NVIDIA collaborent pour accélérer l’ent...,"Fri, 24 Mar 2023 10:51:40 +0000",https://www.actuia.com/?p=42822,
248,Le Fonds Innovation Défense participe à la lev...,"Wed, 22 Mar 2023 12:00:43 +0000",https://www.actuia.com/?p=42791,
249,Selon le baromètre véhicule autonome VEDECOM &...,"Wed, 22 Mar 2023 11:30:26 +0000",https://www.actuia.com/?p=42797,


In [10]:
# Configure Kafka
bootstrap_servers = ['localhost:9092']
topic = 'your_kafka_topic'
producer = KafkaProducer(bootstrap_servers=bootstrap_servers)

# Configure Cassandra
contact_points = ['localhost']
auth_provider = PlainTextAuthProvider(username='cassandra', password='cassandra')
cluster = Cluster(contact_points=contact_points, auth_provider=auth_provider)
session = cluster.connect()


# Envoie du dataFrame vers Kafka
serialized_df = data.to_json().encode('utf-8')  # encode the JSON string to bytes
producer.send(topic, value=serialized_df)

# Le dataFrame est ensuite stocké dans la base de données Cassandra
session.execute("CREATE KEYSPACE IF NOT EXISTS your_keyspace WITH REPLICATION = {'class' : 'SimpleStrategy', 'replication_factor' : 1}")
session.execute("USE your_keyspace")
session.execute("CREATE TABLE IF NOT EXISTS RSS (title text, pubdate text, link text, description text, PRIMARY KEY(feed_id, pubdate)) WITH CLUSTERING ORDER BY (pubdate DESC)")

for _, row in data.iterrows():
    session.execute(f"INSERT INTO RSS (title, pubdate, link, description) VALUES ('{row['title']}', '{row['pubDate']}', '{row['link']}', '{row['description']}')")

In [ ]:
rows = session.execute("SELECT * FROM RSS")
for row in rows:
    print(row.link)

In [ ]:
#Ferme la connection
def close_connection():
    producer.flush()
    producer.close()
    session.shutdown()
    cluster.shutdown()
    producer.flush()
    producer.close()
    session.shutdown()
    cluster.shutdown()

In [ ]:
#Suppression des données dans la basse de données
def delete_data():
    session.execute("DESCRIBE KEYSPACES;")
    session.execute("USE your_keyspace;")
    session.execute("DESCRIBE TABLES;")
    session.execute("TRUNCATE RSS")

In [ ]:
delete_data()